## Elden Ring Steam Review NLP Sentiment Analysis Project - Data Collection

In this project, I will be conducting an overall sentiment analysis and other NLP tasks using review information from select video games off Steam. <b>Steam</b> is an online video game hosting service and is one of the primary outlets to play video games on PC.

After doing some research on how to collect the data to use for this project (let alone any NLP project), I found that there is an existing API to scrape data from Steam, however after running some initial trials of scraping review data, the amount of reviews returned was extremely limited due to the configuration of <i>cursor</i>, which can limit the amount of responses returned to a client in order to hinder large volumes of web traffic (from my understanding of the matter).

While looking into building a scraper to obtain game reviews, I happened to stumble across a readily available scraper using Beauitful Soup that parses the cursor from the current page to use in collection of data from the following page. I've chosen to stick with this scraper implementation just to ensure I get as many reviews as possible. Since we're going to analyze reviews from <b>Elden Ring</b>, I would hope it won't become a nuisance to scrape reviews at only one time. 

So, here is the link to the helpful guide on how to collect video game review data from Steam (where information and code on how to make the scraper comes from): https://andrew-muller.medium.com/scraping-steam-user-reviews-9a43f9e38c92

The goal of this project will be to ultimately try to identify, by conducting various NLP tasks, issues and opinions about select games by reviewing game reviews on Steam. Then, based on the topics and insight found, identify and propose solutions to these problems the communities have fomented. Or, at least identify what the strengths and weaknesses of the games are in the communities' eyes (it really depends on the game).

The tasks that may help us here are sentiment analysis and topic modeling to name a few. 

I felt it would be more beneficial to use games that have released recently, so the content in the reviews is more relevant to the current game state and could relate to how the game gets updated should the developers listen to this outlet of community feedback.




In [148]:
import numpy as np
import pandas as pd
import time
import json
import sys

import requests
import steamreviews
from bs4 import BeautifulSoup

#possible games: yugioh master duel, elden ring

#Project Use Case: taking reviews from these games and perofrming some sentimnet analysis, as well as topic modeling to gget
# a sense of what the reviews are talking about, and how these games can use them to improve their games in future updates.

#we'll try both games and see which gives a better sense of: sentiment, topic modeling. 

If we need to look for certain game app IDs, this is a convenient way to acquire and search for them.

In [149]:
response = requests.get('https://api.steampowered.com/ISteamApps/GetAppList/v2/')
app_ids_df = pd.DataFrame(response.json()['applist']['apps'])
#app_ids_df.to_pickle('../data/app_ids.pkl.gz')
app_ids_df

,appid,name
0,216938,Pieterw test app76 ( 216938 )
1,660010,test2
2,660130,test3
3,1118314,
4,1275822,
...,...,...
140530,1770080,Superfuse
140531,801550,VAIL
140532,1867430,Cygnus Racing League
140533,1712700,Afterlife


Below, we implement the scraper to collect review data on our video game (SSSSS) of choice:

For this project, I will only be analyzing data that is in English.

In [150]:
response = requests.get(url='https://store.steampowered.com/appreviews/1449850?json=1').json()

In [151]:
print(type(response))

<class 'dict'>


In [155]:
def get_reviews(appid, params={'json':1}):
    """
    This function sends a request to return a page e of reviews.
    
    args:
        appid (int): The unique ID of the game to search reviews for
        params (dict): the parameters to pass to the request.get method in order to acquire reviews appropriately
    
    returns:
        response (list): the returned requests object list containing review information on a particular page of reviews
    
    
    
    """
        url = 'https://store.steampowered.com/appreviews/'
        try:
            response = requests.get(url=url+appid, params=params, headers={'User-Agent': 'Mozilla/5.0'})
        except:
            return {'reviews' : []}
        return response.json()

In [163]:
def get_n_reviews(appid, n=100):
    """ 
    This function scrapes several pages of a game's reviews and returns them in a list
    
    args:
        appid (int): The unique ID of the game to search reviews for
        n (int): The amount of reviews per page to acquire
        
    returns:
        reviews (list): The reviews of the game as well as other information regarding a single review
    
    
    """
    reviews = []
    cursor = '*'
    params = {
            'json' : 1,
            'filter' : 'all',
            'language' : 'english', #scraping only English reviews
            'day_range' : 9223372036854775807,
            'review_type' : 'all',
            'purchase_type' : 'all'
            }

    games = ['1449850', '1245620'] #Our game(s) of interest's appids
    
    while n > 0:
        params['cursor'] = cursor.encode()
        params['num_per_page'] = min(100, n)
        n -= 5

        response = get_reviews(appid, params)
        cursor = response['cursor']
        reviews += response['reviews']
        
        print('Current amount of reviews: {}'.format(len(reviews)))
        
        #Specifically chosen points to break the loop so the scraper doesn't end early or run infinitely
        #To elaborate, I have chosen how long the reviews list should be until it should exit the while loop
        #These are estimates taken from the total English reviews of these games on Steam.

        if appid not in games: #Since Edlen Ring has over 240k reviews, but I don't think all of them will be necessary.
            if len(response['reviews']) < 1: break #In effort to get more reviews
                
        elif appid == '1449850':
            if len(reviews) > 20000 or len(response['reviews']) < 1: break #Y-Gi-Oh
                
        elif appid == '1245620':
            if len(reviews) > 240000: break #For Elden Ring, we'll take approximately 200k in case it is too much.

    return reviews

In [158]:
start = time.time()

yugioh_reviews = get_n_reviews(appid='1449850', n=1000000000000)

end = time.time()

print("Time taken to scrape Yu-Gi-Oh! Master Duel reviews: {}".format(end-start))

Current amount of reviews: 100
Current amount of reviews: 200
Current amount of reviews: 300
Current amount of reviews: 400
Current amount of reviews: 500
Current amount of reviews: 600
Current amount of reviews: 700
Current amount of reviews: 800
Current amount of reviews: 900
Current amount of reviews: 1000
Current amount of reviews: 1100
Current amount of reviews: 1200
Current amount of reviews: 1300
Current amount of reviews: 1400
Current amount of reviews: 1500
Current amount of reviews: 1600
Current amount of reviews: 1700
Current amount of reviews: 1800
Current amount of reviews: 1900
Current amount of reviews: 2000
Current amount of reviews: 2100
Current amount of reviews: 2200
Current amount of reviews: 2300
Current amount of reviews: 2400
Current amount of reviews: 2500
Current amount of reviews: 2600
Current amount of reviews: 2700
Current amount of reviews: 2800
Current amount of reviews: 2900
Current amount of reviews: 3000
Current amount of reviews: 3100
Current amount of

In [166]:
yugioh_reviews #voted_up is the attribute for recommended or not.

[{'recommendationid': '108515968',
  'author': {'steamid': '76561198367423267',
   'num_games_owned': 30,
   'num_reviews': 22,
   'playtime_forever': 3120,
   'playtime_last_two_weeks': 81,
   'playtime_at_review': 600,
   'last_played': 1650275950},
  'language': 'english',
  'review': 'How 2 Master Duel \n\n*Play ranked\n*Opponent is using Virtual World\n*I go and cook an entire meal as the guy sets down nonsense \n*I negate his entire combo with one card\n*I perform one combo\n*Opponent forfeits\n\n⠻⣦⡈⠛⢿⡌⢆⢂⠹⣿⣧⡄⡄⠹⡄⢆⡟⢄⠉⢟⣮⣷⢄⠐⢤⡤⢤⣬⣭⣭\n⣷⣤⠙⣶⣦⠙⡌⢻⠂⠘⣿⣷⢹⠄⢳⠘⣿⡟⣿⡄⢳⡆⢳⠛⢢⠈⠉⠛⠛⠛\n⢿⣿⣷⣀⠈⠓⠌⠂⠘⢆⠈⢙⣎⢣⠘⡆⢣⡛⣿⡇⢀⡙⠚⠶⠶⠿⠷⠶⠒⠒\n⣾⠿⣿⣿⣦⣄⣁⣀⣠⡀⠡⠄⠳⡛⢆⠱⠈⢧⣿⡇⠸⠟⠉⠐⠠⠄⠄⠄⣼⣿\n⣿⣷⣬⣉⣛⣛⠛⠿⠿⣧⣷⣦⣄⠘⢿⡆⠡⠘⣀⡇⢠⣶⣷⣮⣤⣤⣴⣿⣿⣿\n⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⡀⢿⡀⠄⠉⡇⢸⣿⣿⣿⣉⣉⣉⣉⣉⣰\n⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡏⣿⣦⡙⠄⠄⠁⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿\n⣿⡙⠿⣿⣿⣿⣿⣿⣿⣿⣿⣿⠿⠁⣿⣿⣷⣌⠄⠄⠘⣿⣿⣿⣿⣿⣿⣿⣿⣿\n⣿⣿⣦⣀⡨⣝⡻⠿⣻⣿⣧⣄⠄⠄⣿⣿⣿⣿⣿⣦⡀⢻⣿⣿⣿⣿⣿⡿⠿⣿\n⣿⣿⣿⣿⡇⢿⡛⢿⣿⣿⣿⣿⣧⠘⣿⣛⣻⣿⣿⣿⣿⣤⡿⢿⠿⣛⡃⠄⣸⣿\n⡹⣿⣿⣿⣿⣼⣷⣶⣝⢿⣿⣿⣿⣧⣿⣿⣿⣿⣿⣿⣿⣿⡿⠟⣿⢇⣈⣼⣿⣿\n⣿⣜⢿⣭⢻⣿⣿⣿⣯⢧⠙⢻⠛⣛⣛⠛⠛⠿⠿⠟⣛⣥⣶⣼⢏⣾⣿⣿⣿⣿\n⣿⣿⣎⠿⣇⢻⣿⣿⣿⡌⢷⣆⢭⣓⣒⣒⣛⣣⣶⠞⣵⣿⣿⣿⡿⣱⣾⡟⣵⣿\n⣿⣿⣿⣤⣻⡘⣿⣿⣿⣷⡜⢻⣮⣛⣛⣟⣫⡿⣛⣴⣿⣿⣿⣿⢃⣿⣿⣷⣿⣿\n⣿⣿⣿⣿⣷⣣⢹⣿⣿⣿⣿⣯⡙⠛⠛⠛⢛⣺⣿⣿⣿⣿⣿⠇⣼⢫⣵⣿⣿',
  'timestamp

We can see we not only get the review text, but some other information on the reviewer, such as the number of games they have on Steam.

In [167]:
yugioh_reviews[0] #The first record in the returned reviews, we can see the reviewer left a cute little picture for us!

#weighted vote score I believe is the score for helpfullness.

{'recommendationid': '108515968',
 'author': {'steamid': '76561198367423267',
  'num_games_owned': 30,
  'num_reviews': 22,
  'playtime_forever': 3120,
  'playtime_last_two_weeks': 81,
  'playtime_at_review': 600,
  'last_played': 1650275950},
 'language': 'english',
 'review': 'How 2 Master Duel \n\n*Play ranked\n*Opponent is using Virtual World\n*I go and cook an entire meal as the guy sets down nonsense \n*I negate his entire combo with one card\n*I perform one combo\n*Opponent forfeits\n\n⠻⣦⡈⠛⢿⡌⢆⢂⠹⣿⣧⡄⡄⠹⡄⢆⡟⢄⠉⢟⣮⣷⢄⠐⢤⡤⢤⣬⣭⣭\n⣷⣤⠙⣶⣦⠙⡌⢻⠂⠘⣿⣷⢹⠄⢳⠘⣿⡟⣿⡄⢳⡆⢳⠛⢢⠈⠉⠛⠛⠛\n⢿⣿⣷⣀⠈⠓⠌⠂⠘⢆⠈⢙⣎⢣⠘⡆⢣⡛⣿⡇⢀⡙⠚⠶⠶⠿⠷⠶⠒⠒\n⣾⠿⣿⣿⣦⣄⣁⣀⣠⡀⠡⠄⠳⡛⢆⠱⠈⢧⣿⡇⠸⠟⠉⠐⠠⠄⠄⠄⣼⣿\n⣿⣷⣬⣉⣛⣛⠛⠿⠿⣧⣷⣦⣄⠘⢿⡆⠡⠘⣀⡇⢠⣶⣷⣮⣤⣤⣴⣿⣿⣿\n⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⡀⢿⡀⠄⠉⡇⢸⣿⣿⣿⣉⣉⣉⣉⣉⣰\n⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡏⣿⣦⡙⠄⠄⠁⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿\n⣿⡙⠿⣿⣿⣿⣿⣿⣿⣿⣿⣿⠿⠁⣿⣿⣷⣌⠄⠄⠘⣿⣿⣿⣿⣿⣿⣿⣿⣿\n⣿⣿⣦⣀⡨⣝⡻⠿⣻⣿⣧⣄⠄⠄⣿⣿⣿⣿⣿⣦⡀⢻⣿⣿⣿⣿⣿⡿⠿⣿\n⣿⣿⣿⣿⡇⢿⡛⢿⣿⣿⣿⣿⣧⠘⣿⣛⣻⣿⣿⣿⣿⣤⡿⢿⠿⣛⡃⠄⣸⣿\n⡹⣿⣿⣿⣿⣼⣷⣶⣝⢿⣿⣿⣿⣧⣿⣿⣿⣿⣿⣿⣿⣿⡿⠟⣿⢇⣈⣼⣿⣿\n⣿⣜⢿⣭⢻⣿⣿⣿⣯⢧⠙⢻⠛⣛⣛⠛⠛⠿⠿⠟⣛⣥⣶⣼⢏⣾⣿⣿⣿⣿\n⣿⣿⣎⠿⣇⢻⣿⣿⣿⡌⢷⣆⢭⣓⣒⣒⣛⣣⣶⠞⣵⣿⣿⣿⡿⣱⣾⡟⣵⣿\n⣿⣿⣿⣤⣻⡘⣿⣿⣿⣷⡜⢻⣮⣛⣛⣟⣫⡿⣛⣴⣿⣿⣿⣿⢃⣿⣿⣷⣿⣿\n⣿⣿⣿⣿⣷⣣⢹⣿⣿⣿⣿⣯⡙⠛⠛⠛⢛⣺⣿⣿⣿⣿⣿⠇⣼⢫⣵⣿⣿',
 'timestamp_created': 

In [170]:
len(yugioh_reviews) #The length represents the total amount of reviews.

19425

Scraping Elden Ring Below: Elden Ring app id: 1245620

In [171]:
start = time.time()

elden_ring_reviews = get_n_reviews(appid='1245620', n=100000000000000)

end = time.time()

print("Time taken to scrape Elden Ring's reviews: {}".format(end-start))

Current amount of reviews: 100
Current amount of reviews: 200
Current amount of reviews: 300
Current amount of reviews: 400
Current amount of reviews: 500
Current amount of reviews: 600
Current amount of reviews: 700
Current amount of reviews: 800
Current amount of reviews: 900
Current amount of reviews: 1000
Current amount of reviews: 1100
Current amount of reviews: 1200
Current amount of reviews: 1300
Current amount of reviews: 1400
Current amount of reviews: 1500
Current amount of reviews: 1600
Current amount of reviews: 1700
Current amount of reviews: 1800
Current amount of reviews: 1900
Current amount of reviews: 2000
Current amount of reviews: 2100
Current amount of reviews: 2200
Current amount of reviews: 2300
Current amount of reviews: 2400
Current amount of reviews: 2500
Current amount of reviews: 2600
Current amount of reviews: 2700
Current amount of reviews: 2800
Current amount of reviews: 2900
Current amount of reviews: 3000
Current amount of reviews: 3100
Current amount of

Current amount of reviews: 25300
Current amount of reviews: 25400
Current amount of reviews: 25500
Current amount of reviews: 25600
Current amount of reviews: 25700
Current amount of reviews: 25800
Current amount of reviews: 25900
Current amount of reviews: 26000
Current amount of reviews: 26100
Current amount of reviews: 26200
Current amount of reviews: 26300
Current amount of reviews: 26400
Current amount of reviews: 26500
Current amount of reviews: 26600
Current amount of reviews: 26700
Current amount of reviews: 26800
Current amount of reviews: 26900
Current amount of reviews: 27000
Current amount of reviews: 27100
Current amount of reviews: 27200
Current amount of reviews: 27300
Current amount of reviews: 27400
Current amount of reviews: 27500
Current amount of reviews: 27600
Current amount of reviews: 27700
Current amount of reviews: 27800
Current amount of reviews: 27900
Current amount of reviews: 28000
Current amount of reviews: 28100
Current amount of reviews: 28200
Current am

Current amount of reviews: 50200
Current amount of reviews: 50300
Current amount of reviews: 50400
Current amount of reviews: 50500
Current amount of reviews: 50600
Current amount of reviews: 50700
Current amount of reviews: 50800
Current amount of reviews: 50900
Current amount of reviews: 51000
Current amount of reviews: 51100
Current amount of reviews: 51200
Current amount of reviews: 51300
Current amount of reviews: 51400
Current amount of reviews: 51500
Current amount of reviews: 51600
Current amount of reviews: 51700
Current amount of reviews: 51800
Current amount of reviews: 51900
Current amount of reviews: 52000
Current amount of reviews: 52100
Current amount of reviews: 52200
Current amount of reviews: 52300
Current amount of reviews: 52400
Current amount of reviews: 52500
Current amount of reviews: 52600
Current amount of reviews: 52700
Current amount of reviews: 52800
Current amount of reviews: 52900
Current amount of reviews: 53000
Current amount of reviews: 53100
Current am

Current amount of reviews: 75100
Current amount of reviews: 75200
Current amount of reviews: 75300
Current amount of reviews: 75400
Current amount of reviews: 75500
Current amount of reviews: 75600
Current amount of reviews: 75700
Current amount of reviews: 75800
Current amount of reviews: 75900
Current amount of reviews: 76000
Current amount of reviews: 76100
Current amount of reviews: 76200
Current amount of reviews: 76300
Current amount of reviews: 76400
Current amount of reviews: 76500
Current amount of reviews: 76600
Current amount of reviews: 76700
Current amount of reviews: 76800
Current amount of reviews: 76900
Current amount of reviews: 77000
Current amount of reviews: 77100
Current amount of reviews: 77200
Current amount of reviews: 77300
Current amount of reviews: 77400
Current amount of reviews: 77500
Current amount of reviews: 77600
Current amount of reviews: 77700
Current amount of reviews: 77800
Current amount of reviews: 77900
Current amount of reviews: 78000
Current am

Current amount of reviews: 100000
Current amount of reviews: 100100
Current amount of reviews: 100200
Current amount of reviews: 100300
Current amount of reviews: 100400
Current amount of reviews: 100500
Current amount of reviews: 100600
Current amount of reviews: 100700
Current amount of reviews: 100800
Current amount of reviews: 100900
Current amount of reviews: 101000
Current amount of reviews: 101100
Current amount of reviews: 101200
Current amount of reviews: 101300
Current amount of reviews: 101400
Current amount of reviews: 101500
Current amount of reviews: 101600
Current amount of reviews: 101700
Current amount of reviews: 101800
Current amount of reviews: 101900
Current amount of reviews: 102000
Current amount of reviews: 102100
Current amount of reviews: 102200
Current amount of reviews: 102300
Current amount of reviews: 102400
Current amount of reviews: 102500
Current amount of reviews: 102600
Current amount of reviews: 102700
Current amount of reviews: 102800
Current amount

Current amount of reviews: 124100
Current amount of reviews: 124200
Current amount of reviews: 124300
Current amount of reviews: 124400
Current amount of reviews: 124500
Current amount of reviews: 124600
Current amount of reviews: 124700
Current amount of reviews: 124800
Current amount of reviews: 124900
Current amount of reviews: 125000
Current amount of reviews: 125100
Current amount of reviews: 125200
Current amount of reviews: 125300
Current amount of reviews: 125400
Current amount of reviews: 125500
Current amount of reviews: 125600
Current amount of reviews: 125700
Current amount of reviews: 125800
Current amount of reviews: 125900
Current amount of reviews: 126000
Current amount of reviews: 126100
Current amount of reviews: 126200
Current amount of reviews: 126300
Current amount of reviews: 126400
Current amount of reviews: 126500
Current amount of reviews: 126600
Current amount of reviews: 126700
Current amount of reviews: 126800
Current amount of reviews: 126900
Current amount

Current amount of reviews: 148200
Current amount of reviews: 148300
Current amount of reviews: 148400
Current amount of reviews: 148500
Current amount of reviews: 148600
Current amount of reviews: 148700
Current amount of reviews: 148800
Current amount of reviews: 148900
Current amount of reviews: 149000
Current amount of reviews: 149100
Current amount of reviews: 149200
Current amount of reviews: 149300
Current amount of reviews: 149400
Current amount of reviews: 149500
Current amount of reviews: 149600
Current amount of reviews: 149700
Current amount of reviews: 149800
Current amount of reviews: 149900
Current amount of reviews: 150000
Current amount of reviews: 150100
Current amount of reviews: 150200
Current amount of reviews: 150300
Current amount of reviews: 150400
Current amount of reviews: 150500
Current amount of reviews: 150600
Current amount of reviews: 150700
Current amount of reviews: 150800
Current amount of reviews: 150900
Current amount of reviews: 151000
Current amount

Current amount of reviews: 172300
Current amount of reviews: 172400
Current amount of reviews: 172500
Current amount of reviews: 172600
Current amount of reviews: 172700
Current amount of reviews: 172800
Current amount of reviews: 172900
Current amount of reviews: 173000
Current amount of reviews: 173100
Current amount of reviews: 173200
Current amount of reviews: 173300
Current amount of reviews: 173400
Current amount of reviews: 173500
Current amount of reviews: 173600
Current amount of reviews: 173700
Current amount of reviews: 173800
Current amount of reviews: 173900
Current amount of reviews: 174000
Current amount of reviews: 174100
Current amount of reviews: 174200
Current amount of reviews: 174300
Current amount of reviews: 174400
Current amount of reviews: 174500
Current amount of reviews: 174600
Current amount of reviews: 174700
Current amount of reviews: 174800
Current amount of reviews: 174900
Current amount of reviews: 175000
Current amount of reviews: 175100
Current amount

Current amount of reviews: 196400
Current amount of reviews: 196500
Current amount of reviews: 196600
Current amount of reviews: 196700
Current amount of reviews: 196800
Current amount of reviews: 196900
Current amount of reviews: 197000
Current amount of reviews: 197100
Current amount of reviews: 197200
Current amount of reviews: 197300
Current amount of reviews: 197400
Current amount of reviews: 197500
Current amount of reviews: 197600
Current amount of reviews: 197700
Current amount of reviews: 197800
Current amount of reviews: 197900
Current amount of reviews: 198000
Current amount of reviews: 198100
Current amount of reviews: 198200
Current amount of reviews: 198300
Current amount of reviews: 198400
Current amount of reviews: 198500
Current amount of reviews: 198600
Current amount of reviews: 198700
Current amount of reviews: 198800
Current amount of reviews: 198900
Current amount of reviews: 199000
Current amount of reviews: 199100
Current amount of reviews: 199200
Current amount

Current amount of reviews: 220500
Current amount of reviews: 220600
Current amount of reviews: 220700
Current amount of reviews: 220800
Current amount of reviews: 220900
Current amount of reviews: 221000
Current amount of reviews: 221100
Current amount of reviews: 221200
Current amount of reviews: 221300
Current amount of reviews: 221400
Current amount of reviews: 221500
Current amount of reviews: 221600
Current amount of reviews: 221700
Current amount of reviews: 221800
Current amount of reviews: 221900
Current amount of reviews: 222000
Current amount of reviews: 222100
Current amount of reviews: 222200
Current amount of reviews: 222300
Current amount of reviews: 222400
Current amount of reviews: 222500
Current amount of reviews: 222600
Current amount of reviews: 222700
Current amount of reviews: 222800
Current amount of reviews: 222900
Current amount of reviews: 223000
Current amount of reviews: 223100
Current amount of reviews: 223200
Current amount of reviews: 223300
Current amount

In [172]:
len(elden_ring_reviews)

240100

In [173]:
elden_ring_reviews[0]

{'recommendationid': '112052982',
 'author': {'steamid': '76561197971854668',
  'num_games_owned': 180,
  'num_reviews': 3,
  'playtime_forever': 14266,
  'playtime_last_two_weeks': 2508,
  'playtime_at_review': 977,
  'last_played': 1650576902},
 'language': 'english',
 'review': "First time with a Dark Souls type game and loving it. I am 52 so apparently it's never too late to get good.",
 'timestamp_created': 1647106589,
 'timestamp_updated': 1647106589,
 'voted_up': True,
 'votes_up': 13373,
 'votes_funny': 1063,
 'weighted_vote_score': '0.969000637531280518',
 'comment_count': 0,
 'steam_purchase': True,
 'received_for_free': False,
 'written_during_early_access': False}

In [174]:
#Checking the size of the elden ring reviews file to ensure it isn't too big to export somewhere

print('Size in MB: {}'.format(sys.getsizeof(elden_ring_reviews) / 1000000))

Size in MB: 2.1322


Now that we have the reviews data for our games of choice, we can start analyzing them!

We'll do this in the next notebook.

In [175]:
print(type(yugioh_reviews))
print(type(elden_ring_reviews))

<class 'list'>
<class 'list'>


In [176]:
yugioh_df = pd.DataFrame(yugioh_reviews)
elden_ring_df = pd.DataFrame(elden_ring_reviews)

In [177]:
yugioh_df.head()

,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access
0,108515968,"{'steamid': '76561198367423267', 'num_games_ow...",english,How 2 Master Duel \n\n*Play ranked\n*Opponent ...,1642835616,1643181237,True,1565,1192,0.964541196823120117,38,False,False,False
1,109328043,"{'steamid': '76561198066637460', 'num_games_ow...",english,writing this review cause my opponent still ha...,1643815800,1643815858,True,686,798,0.952718019485473633,0,False,False,False
2,109769548,"{'steamid': '76561198048179366', 'num_games_ow...",english,My favorite part was when it was my turn.,1644305797,1644305797,True,335,237,0.94757610559463501,0,False,False,False
3,108403889,"{'steamid': '76561198192657635', 'num_games_ow...",english,If you are willing to spend half of your lifet...,1642702485,1642702485,True,955,529,0.946273565292358398,0,False,False,False
4,108301458,"{'steamid': '76561198451668045', 'num_games_ow...",english,absolute blast! cannot wait to perform an FTK ...,1642557885,1642557885,True,1022,728,0.944011449813842773,15,False,False,False


In [178]:
elden_ring_df.head()

,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access
0,112052982,"{'steamid': '76561197971854668', 'num_games_ow...",english,First time with a Dark Souls type game and lov...,1647106589,1647106589,True,13373,1063,0.969000637531280518,0,True,False,False
1,111180228,"{'steamid': '76561198142415989', 'num_games_ow...",english,"The note said ""chest ahead"" at the edge of a c...",1645984692,1645984692,True,10448,6200,0.968308687210083008,144,True,False,False
2,111834761,"{'steamid': '76561197996565477', 'num_games_ow...",english,you can hug a woman,1646786974,1646786974,True,7580,2737,0.965730786323547363,81,False,False,False
3,111321870,"{'steamid': '76561197977618010', 'num_games_ow...",english,If you rush you can make it the hardest Souls ...,1646160050,1646160090,True,4839,63,0.96507793664932251,0,True,False,False
4,112422103,"{'steamid': '76561198191736422', 'num_games_ow...",english,First game in a long time where you get exactl...,1647625058,1647625058,True,4968,80,0.963619053363800049,0,True,False,False


In [180]:
yugioh_df.tail()

,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access
19420,108301173,"{'steamid': '76561198821535831', 'num_games_ow...",english,time to spend all my money on a play set of to...,1642557459,1642557459,True,0,0,0,0,False,False,False
19421,108301044,"{'steamid': '76561198025015762', 'num_games_ow...",english,"10/10 Masterpiece, Best Yugioh Game ever.",1642557274,1642557274,True,0,0,0,0,False,False,False
19422,108300882,"{'steamid': '76561198035780615', 'num_games_ow...",english,"Love this game, can't wait to play it.",1642557057,1642557057,True,0,0,0,0,False,False,False
19423,108300245,"{'steamid': '76561198150273939', 'num_games_ow...",english,This is going to be Game of the Year i just kn...,1642556098,1642556098,True,0,0,0,0,False,False,False
19424,108300042,"{'steamid': '76561198069508307', 'num_games_ow...",english,It is down for maintenance so you cant play yet,1642555794,1642555794,True,0,0,0,0,False,False,False


In [181]:
elden_ring_df.tail()

,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access
240095,110924825,"{'steamid': '76561198135133370', 'num_games_ow...",english,ANJEEEEEEENKZZZZ,1645755247,1645983529,True,0,0,0,0,True,False,False
240096,110924824,"{'steamid': '76561198028249807', 'num_games_ow...",english,"you don't have the right, O you don't have the...",1645755246,1646380771,True,0,0,0,0,False,False,False
240097,110924809,"{'steamid': '76561198005170638', 'num_games_ow...",english,The game refuses to take me beyond the initial...,1645755235,1645755235,False,0,0,0,0,True,False,False
240098,110924807,"{'steamid': '76561197960895660', 'num_games_ow...",english,Ridiculous 2022 and no widescreen support. WTF...,1645755233,1647345225,True,0,0,0,0,False,False,False
240099,110924800,"{'steamid': '76561198070617836', 'num_games_ow...",english,"Honestly, the game starts to fall apart in the...",1645755228,1647254252,False,0,0,0,0,True,False,False


In [ ]:
yugioh_df.to_csv('yugioh_reviews.csv')
elden_ring_df.to_csv('elden_ring_reviews.csv')

In the next notebook, we'll begin cleaning and analyzing the reviews for Elden Ring. I scraped the reviews for Yu-Gi-Oh! Master Duel as well primarily as a back-up to the Elden Ring reviews in case the sheer amount of reviews for that game would be too cumbersome for my computer.